In [47]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-
import numpy as np
import cv2
import glob

import json
import re
import time
import os

face_cascade = cv2.CascadeClassifier(r'C:\Users\rlatn\AppData\Local\Programs\Python\Python37\haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

#Firebase database 인증 및 앱 초기화
cred = credentials.Certificate(r'C:/Users/rlatn/Project_image_db/aicctv-8f5ac-firebase-adminsdk-n59g4-2e8482cb47.json')
firebase_admin.initialize_app(cred,{
    'databaseURL' : 'https://aicctv-8f5ac.firebaseio.com/'})

ID = "00gpwls00" # 이용자마다 다르게 코딩해야 하는 부분
ref = db.reference(ID+"/PhotoLink") #db 위치 지정

In [52]:
#지정한 경로에 있는 폴더마다 사진 검출
 
#사진에 얼굴이 1개인 사진의 개수 저장하는 변수
#얼굴 1개인 사진만 저장하는 폴더에 이전까지 몇개의 사진 저장되어있는지 나타내는 변수
count = 0   

while True:
    
    dic=ref.get() # dict object
    tmp_str=json.dumps(dic, ensure_ascii=False)[1:-1] # 한글을 포함해 string으로 변환 후 맨 앞과 뒤의 {} 제거
    tmp=re.sub(r': {[^}]*}', '', tmp_str) #  쉼표와 이름을 제외하고 모든 문자 삭제
    people=tmp.split(", ") # 사진에 등록된 사람들 이름만 추출   
    
    for i in range (len(people)):
        person_ref=db.reference(ID+"/PhotoLink/"+people[i][1:-1])
        
        #DB에서 컴퓨터로 다운로드 된 사진의 개수
        DownCount_ref=person_ref.child('DownCount')
        DownCount=DownCount_ref.get()
        
        #사진 저장된 경로 outpath  
        outpath="C:/"+ID+"/"+people[i][1:-1]+"/"
        #예) 'C:/00gpwls00/수영/'
        
        #outpath에 저장되어 있는 사진들 불러오기
        images  = glob.glob(outpath+'**')

        #images에 있는 파일 이름 출력
        #print(images)
        
        #컴퓨터에 다운로드 된 사진들 중 얼굴 검출이 안된 사진이 있다면
        if(DownCount != count):
            for j in range(count, DownCount+1):
                #업데이트 해야하는 사진을 img로 받아오기
                img = images[j]
        
                img = cv2.imread(img)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                face_count = len(faces) 

                #얼굴 개수 출력
                #print("얼굴 개수:",face_count)
                
                #이름 형식 = C:/00gpwls00/수연\20200315_033830_1.png -> 20200315_033830_1
                filename = os.path.basename(img)

                #검출된 얼굴이 1개인 사진만 따로 다른 경로의 파일에 저장
                if(face_count == 1):
                    #검출 얼굴 1개인 사진의 개수 나타내는 변수 count
                    count += 1
                    name += 1
                    cv2.imwrite(filename, img)
                    print("저장 완료")

            
    #위에서 설정한 firebase의 database에 검출한 얼굴이 1개인 사진 개수 update
    ref.update({'검출 얼굴 1개인 사진 개수' : count})   
    
    # 60초에 한번씩만 확인 및 다운로드
    time.sleep(60)


error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
